In [1]:
import torch
import numpy as np
import plotly.graph_objects as go
from collections import defaultdict
import torch.nn.functional as F
import json
import os
from typing import Dict, Tuple, List, Optional
from transformers import AutoModelForCausalLM, AutoTokenizer
import math

In [2]:
def get_device():
    """Get the optimal available device"""
    if torch.cuda.is_available():
        device = torch.device("cuda:0")
        # Enable TF32 for better performance on Ampere GPUs (A100, A6000, etc)
        torch.backends.cuda.matmul.allow_tf32 = True
        torch.backends.cudnn.allow_tf32 = True
        # Set memory allocation settings
        torch.cuda.empty_cache()
        # Enable CUDNN benchmarking for better performance
        torch.backends.cudnn.benchmark = True
    else:
        device = torch.device("cpu")
    return device

In [3]:
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-moe-16b-base",
                                             trust_remote_code=True,
                                             torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-moe-16b-base", trust_remote_code=True)
model.eval()

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

DeepseekForCausalLM(
  (model): DeepseekModel(
    (embed_tokens): Embedding(102400, 2048)
    (layers): ModuleList(
      (0): DeepseekDecoderLayer(
        (self_attn): DeepseekSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): DeepseekRotaryEmbedding()
        )
        (mlp): DeepseekMLP(
          (gate_proj): Linear(in_features=2048, out_features=10944, bias=False)
          (up_proj): Linear(in_features=2048, out_features=10944, bias=False)
          (down_proj): Linear(in_features=10944, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): DeepseekRMSNorm()
        (post_attention_layernorm): DeepseekRMSNorm()
      )
      (1-27): 27 x DeepseekDecod

In [4]:
# class MOEExpertLens:
#     def __init__(self, state_dict: Dict[str, torch.Tensor], tokenizer, device=None):
#         """Initialize the MoE Expert analyzer."""
#         self.device = device if device is not None else get_device()
#         self.state_dict = {k: v.to(self.device) for k, v in state_dict.items()}
#         self.tokenizer = tokenizer
#         self.hidden_size = self.state_dict["model.embed_tokens.weight"].shape[1]
#         self.vocab_size = self.state_dict["model.embed_tokens.weight"].shape[0]
#         # Get model dtype from embeddings
#         self.dtype = self.state_dict["model.embed_tokens.weight"].dtype

#     def _process_expert(self, layer_idx: int, expert_idx: int, hidden_state: torch.Tensor) -> torch.Tensor:
#         """Process hidden state through an expert's weights."""
#         # Get expert weights and ensure dtype match
#         gate_proj = self.state_dict[f"model.layers.{layer_idx}.mlp.experts.{expert_idx}.gate_proj.weight"]
#         up_proj = self.state_dict[f"model.layers.{layer_idx}.mlp.experts.{expert_idx}.up_proj.weight"]
#         down_proj = self.state_dict[f"model.layers.{layer_idx}.mlp.experts.{expert_idx}.down_proj.weight"]

#         hidden_state = hidden_state.to(self.dtype)
        
#         # Apply MLPs sequentially while maintaining batch and sequence dimensions
#         gate_output = F.silu(F.linear(hidden_state, gate_proj))
#         up_output = F.linear(hidden_state, up_proj)
        
#         # Element-wise multiplication and final projection
#         x = gate_output * up_output
#         return F.linear(x, down_proj)

#     def _get_router_output(self, layer_idx: int, hidden_state: torch.Tensor) -> torch.Tensor:
#         """Get router logits for a layer."""
#         router_weights = self.state_dict[f"model.layers.{layer_idx}.mlp.gate.weight"]
#         hidden_state = hidden_state.to(self.dtype)
#         return F.linear(hidden_state, router_weights)

#     def _process_attention(self, layer_idx: int, hidden_state: torch.Tensor, 
#                          attention_mask: Optional[torch.Tensor] = None) -> torch.Tensor:
#         """Process hidden state through self-attention while preserving token-wise information."""
#         q_proj = self.state_dict[f"model.layers.{layer_idx}.self_attn.q_proj.weight"]
#         k_proj = self.state_dict[f"model.layers.{layer_idx}.self_attn.k_proj.weight"]
#         v_proj = self.state_dict[f"model.layers.{layer_idx}.self_attn.v_proj.weight"]
#         o_proj = self.state_dict[f"model.layers.{layer_idx}.self_attn.o_proj.weight"]

#         # Ensure hidden state has correct dtype
#         hidden_state = hidden_state.to(self.dtype)

#         # Get shapes
#         batch_size, seq_len, hidden_dim = hidden_state.shape
#         num_heads = 32  # DeepSeek specific
#         head_dim = hidden_dim // num_heads

#         # Compute QKV with shape preservation
#         q = F.linear(hidden_state, q_proj).view(batch_size, seq_len, num_heads, head_dim)
#         k = F.linear(hidden_state, k_proj).view(batch_size, seq_len, num_heads, head_dim)
#         v = F.linear(hidden_state, v_proj).view(batch_size, seq_len, num_heads, head_dim)

#         # Transpose for attention computation
#         q = q.transpose(1, 2)  # [batch, num_heads, seq_len, head_dim]
#         k = k.transpose(1, 2)
#         v = v.transpose(1, 2)

#         # Compute attention scores
#         # Cast to float32 for better numerical stability in attention computation
#         q_float = q.float()
#         k_float = k.float()
#         v_float = v.float()

#         attn_weights = torch.matmul(q_float, k_float.transpose(-2, -1)) / math.sqrt(head_dim)
        
#         # Apply causal mask
#         causal_mask = torch.triu(torch.ones(seq_len, seq_len, dtype=torch.bool, device=self.device), diagonal=1)
#         attn_weights.masked_fill_(causal_mask, float('-inf'))
        
#         if attention_mask is not None:
#             attn_weights = attn_weights + attention_mask

#         attn_weights = F.softmax(attn_weights, dim=-1)
        
#         # Apply attention to values
#         context = torch.matmul(attn_weights, v_float)
        
#         # Convert back to original dtype
#         context = context.to(self.dtype)
        
#         # Reshape and project to output
#         context = context.transpose(1, 2).contiguous().view(batch_size, seq_len, hidden_dim)
#         return F.linear(context, o_proj)

#     def _apply_layer_norm(self, layer_idx: int, hidden_state: torch.Tensor, norm_type: str) -> torch.Tensor:
#         """Apply layer normalization while maintaining token information."""
#         weight = self.state_dict[f"model.layers.{layer_idx}.{norm_type}.weight"]
#         hidden_state = hidden_state.to(self.dtype)
#         return F.layer_norm(hidden_state, (self.hidden_size,), weight=weight)

#     def _project_to_vocab(self, hidden_state: torch.Tensor) -> torch.Tensor:
#         """Project hidden state to vocabulary space while preserving token-wise information."""
#         lm_head_weights = self.state_dict["lm_head.weight"]
#         hidden_state = hidden_state.to(self.dtype)
#         return F.linear(hidden_state, lm_head_weights)

#     def analyze_text(self, input_ids: torch.Tensor) -> Dict:
#         """Analyze text through expert lens with proper token state propagation."""
#         batch_size, seq_len = input_ids.shape
#         # Initialize hidden states from embeddings
#         hidden_states = self.state_dict["model.embed_tokens.weight"][input_ids]
#         results = {}

#         # Create causal attention mask (in float32 for numerical stability)
#         causal_mask = torch.triu(torch.ones(seq_len, seq_len, dtype=torch.bool, device=self.device), diagonal=1)
#         attention_mask = torch.zeros(batch_size, 1, seq_len, seq_len, dtype=torch.float32, device=self.device)
#         attention_mask.masked_fill_(causal_mask, float('-inf'))

#         for layer_idx in range(1, 28):  # Layers 1-27
#             layer_results = {"tokens": {}}
            
#             # Apply input layer norm while preserving token information
#             normed_states = self._apply_layer_norm(layer_idx, hidden_states, "input_layernorm")
            
#             # Process through attention mechanism
#             attn_output = self._process_attention(layer_idx, normed_states, attention_mask)
#             hidden_states = hidden_states + attn_output  # Residual connection
            
#             # Post-attention layer norm
#             normed_states = self._apply_layer_norm(layer_idx, hidden_states, "post_attention_layernorm")
            
#             # Get router decisions for each token
#             router_logits = self._get_router_output(layer_idx, normed_states)
            
#             # Process each token position separately
#             for pos in range(seq_len):
#                 token = self.tokenizer.decode([input_ids[0, pos].item()])
#                 token_state = normed_states[:, pos:pos+1]  # Keep batch dimension
                
#                 # Get top-k experts for this token
#                 top_k_experts = torch.topk(router_logits[:, pos], k=7, dim=-1)
#                 expert_weights = F.softmax(top_k_experts.values, dim=-1)
                
#                 expert_outputs = []
#                 # Process through selected experts
#                 for idx, expert_idx in enumerate(top_k_experts.indices[0]):
#                     expert_output = self._process_expert(layer_idx, expert_idx.item(), token_state)
#                     weight = expert_weights[0, idx].item()
                    
#                     # Project expert output to vocab space for analysis
#                     logits = self._project_to_vocab(expert_output)
#                     top_tokens = torch.topk(logits.squeeze(1), k=5)
                    
#                     expert_outputs.append({
#                         "expert_id": expert_idx.item(),
#                         "weight": weight,
#                         "top_tokens": [
#                             (self.tokenizer.decode([idx.item()]), prob.item())
#                             for idx, prob in zip(top_tokens.indices[0], 
#                                                F.softmax(top_tokens.values[0], dim=-1))
#                         ]
#                     })
                
#                 layer_results["tokens"][token] = {
#                     "position": pos,
#                     "expert_outputs": expert_outputs
#                 }
                
#                 # Update hidden states for this token with weighted expert outputs
#                 token_output = torch.zeros_like(token_state)
#                 for expert_out in expert_outputs:
#                     expert_idx = expert_out["expert_id"]
#                     weight = expert_out["weight"]
#                     expert_output = self._process_expert(layer_idx, expert_idx, token_state)
#                     token_output += weight * expert_output
                
#                 # Update the hidden states for this position
#                 hidden_states[:, pos:pos+1] = hidden_states[:, pos:pos+1] + token_output

#             results[f"layer_{layer_idx}"] = layer_results

#         return results

### hooked version

In [21]:
class MOELens:
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
        self.activations = defaultdict(dict)
        self.hook_handles = []
        self.setup_hooks()

    def setup_hooks(self):
        def get_gate_hook(layer_idx):
            def hook(module, inp, out):
                if isinstance(out, tuple):
                    topk_idx, topk_weight, _ = out
                    
                    # Get hidden states from input
                    hidden_states = inp[0]
                    batch_size, seq_len, hidden_dim = hidden_states.shape
                    
                    # Project to vocab space to get token predictions
                    with torch.no_grad():
                        # Get expert outputs
                        expert_outputs = {}
                        for expert_idx in range(module.n_routed_experts):
                            if hasattr(self.model.model.layers[layer_idx].mlp, 'experts'):
                                expert = self.model.model.layers[layer_idx].mlp.experts[expert_idx]
                                expert_output = expert(hidden_states.view(-1, hidden_dim))
                                # Project to vocabulary space
                                logits = self.model.lm_head(expert_output)
                                top_tokens = torch.topk(logits, k=5, dim=-1)
                                expert_outputs[expert_idx] = {
                                    'token_ids': top_tokens.indices,
                                    'probs': torch.softmax(top_tokens.values, dim=-1)
                                }

                    self.activations[f'layer_{layer_idx}'] = {
                        'router_weights': topk_weight.detach(),
                        'router_indices': topk_idx.detach(),
                        'expert_outputs': expert_outputs
                    }
            return hook

        for i, layer in enumerate(self.model.model.layers):
            if hasattr(layer.mlp, 'gate'):
                handle = layer.mlp.gate.register_forward_hook(get_gate_hook(i))
                self.hook_handles.append(handle)

    def analyze_text(self, input_ids: torch.Tensor) -> dict:
        self.activations.clear()
        
        with torch.no_grad():
            outputs = self.model(input_ids)

        results = {}
        for layer_name, acts in self.activations.items():
            layer_results = {"tokens": {}}
            
            for pos in range(input_ids.shape[1]):
                token = self.tokenizer.decode([input_ids[0, pos].item()])
                expert_info = []
                
                if 'router_indices' in acts and 'router_weights' in acts:
                    indices = acts['router_indices'][pos]
                    weights = acts['router_weights'][pos]
                    expert_outputs = acts['expert_outputs']
                    
                    for idx, weight in zip(indices, weights):
                        expert_id = idx.item()
                        if expert_id in expert_outputs:
                            expert_output = expert_outputs[expert_id]
                            top_tokens = [
                                (self.tokenizer.decode([token_id.item()]), prob.item())
                                for token_id, prob in zip(
                                    expert_output['token_ids'][pos],
                                    expert_output['probs'][pos]
                                )
                            ]
                        else:
                            top_tokens = []
                            
                        expert_info.append({
                            "expert_id": expert_id,
                            "weight": weight.item(),
                            "top_tokens": top_tokens
                        })
                
                layer_results["tokens"][token] = {
                    "position": pos,
                    "expert_outputs": expert_info
                }
            
            results[layer_name] = layer_results

        return results

    def remove_hooks(self):
        for handle in self.hook_handles:
            handle.remove()
        self.hook_handles = []

In [6]:
def visualize_layer_analysis(tokenizer, results: Dict, token_position: int, input_text: str):
    """
    Creates a plotly visualization of expert activations across layers for a specific token.
    Shows all 64 experts with zero weights for non-selected experts.
    """
    # Create lists to store data 
    layer_nums = []
    expert_ids = []
    weights = []
    hover_texts = []
    
    total_experts = 64  # Total number of experts in the model
    
    # Extract token we're visualizing by tokenizing input text first
    tokens = tokenizer.encode(input_text)
    token = tokenizer.decode([tokens[token_position]])  # Get tokenized token
    print(f"Visualizing token: {token}")
        
    for layer_idx in range(1, 28):  # Layers 1-27
        layer_data = results[f"layer_{layer_idx}"]
        token_data = [data for data in layer_data["tokens"].values() 
                     if data["position"] == token_position][0]
        
        # Create a mapping of expert_id to its data for this layer
        expert_map = {exp["expert_id"]: exp for exp in token_data["expert_outputs"]}
        
        # Go through all possible experts
        for expert_id in range(total_experts):
            layer_nums.append(layer_idx)
            expert_ids.append(expert_id)
            
            if expert_id in expert_map:
                # Expert was selected
                expert_data = expert_map[expert_id]
                weight = expert_data["weight"]
                top_tokens_text = "<br>".join([
                    f"{token}: {prob:.3f}" 
                    for token, prob in expert_data["top_tokens"][:5]
                ])
                hover_text = f"Layer: {layer_idx}<br>Expert: {expert_id}<br>Weight: {weight:.3f}<br>Top tokens:<br>{top_tokens_text}"
                hover_texts.append(hover_text)
            else:
                # Expert was not selected
                weight = 0
                hover_texts.append(None)  # No hover text for unselected experts
            
            weights.append(weight)
    
    # Create plotly heatmap
    fig = go.Figure(data=go.Scatter(
        x=expert_ids,
        y=layer_nums,
        mode='markers',
        marker=dict(
            size=9,
            color=weights,
            colorscale=[
                [0, 'rgba(24, 21, 23, 0.8)'],  # Very dark/transparent for zero weights
                [0.0001, 'rgb(68,1,84)'],      # Start of Viridis colorscale
                [1, 'rgb(253,231,37)']         # End of Viridis colorscale
            ],
            cmin=0,  # Set minimum of color scale to 0
            cmax=1,  # Set maximum of color scale to 1
            showscale=True,
            colorbar=dict(
                title='Weight',
                tickmode='linear',
                tick0=0,
                dtick=0.2
            ),
        ),
        text=hover_texts,
        hoverinfo='text',
        hovertemplate='%{text}<extra></extra>',  # Only show hover when text exists
    ))
    
    # Update layout with dark theme
    fig.update_layout(
        template='plotly_dark',
        title=f'Expert Activations for Token "{token}" at Position {token_position}',
        xaxis_title='Expert ID',
        yaxis_title='Layer',
        yaxis=dict(autorange='reversed'),  # Reverse y-axis to have layer 1 at top
        width=1200,
        height=800,
        showlegend=False,
        plot_bgcolor='black',
        paper_bgcolor='black'
    )
    
    # Add grid lines
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='rgba(128, 128, 128, 0.2)', 
                     range=[-1, total_experts])
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='rgba(128, 128, 128, 0.2)')
    
    fig.show()

In [7]:
# def plot_enhanced_logit_lens(model_outputs, tokenizer, input_text, position=None):
#     """
#     Creates enhanced visualization of logit lens analysis using plotly.
    
#     Args:
#         model_outputs: Dictionary containing layer outputs and logits
#         tokenizer: The model's tokenizer
#         input_text: The input text being analyzed
#         position: Optional specific position to analyze
#     """
#     import plotly.graph_objects as go
#     from plotly.subplots import make_subplots
#     import numpy as np
    
#     # Extract data
#     layers = []
#     tokens = []
#     logits = []
#     ranks = []
    
#     # Process each layer's outputs
#     for layer_idx, layer_data in sorted(model_outputs.items()):
#         if not layer_idx.startswith('layer_'):
#             continue
            
#         layer_num = int(layer_idx.split('_')[1])
#         if position is not None:
#             token_data = [data for data in layer_data["tokens"].values() 
#                          if data["position"] == position][0]
#         else:
#             token_data = next(iter(layer_data["tokens"].values()))
            
#         # Get top 5 predictions for this layer
#         top_predictions = []
#         top_logits = []
#         for exp_output in token_data["expert_outputs"]:
#             for tok, prob in exp_output["top_tokens"]:
#                 if tok not in top_predictions:
#                     top_predictions.append(tok)
#                     top_logits.append(prob)
#                 if len(top_predictions) >= 5:
#                     break
#             if len(top_predictions) >= 5:
#                 break
                
#         layers.append(f"Layer {layer_num}")
#         tokens.extend([t for t in top_predictions if t not in tokens])
#         logits.append(top_logits)
        
#     # Create matrix of logits
#     logit_matrix = np.zeros((len(layers), len(tokens)))
#     for i, layer_logits in enumerate(logits):
#         for j, token in enumerate(tokens[:len(layer_logits)]):
#             logit_matrix[i, j] = layer_logits[j]
            
#     # Create subplots
#     fig = make_subplots(rows=2, cols=1, 
#                        subplot_titles=("Token Logits Across Layers", 
#                                      "Top Token Rankings"),
#                        vertical_spacing=0.15)
    
#     # Add heatmap for logit values
#     fig.add_trace(
#         go.Heatmap(
#             z=logit_matrix,
#             x=tokens,
#             y=layers,
#             colorscale='Viridis',
#             text=[[f"{val:.2f}" for val in row] for row in logit_matrix],
#             texttemplate="%{text}",
#             textfont={"size":10},
#             showscale=True,
#             name="Logits"
#         ),
#         row=1, col=1
#     )
    
#     # Add scatter plot for rankings
#     rank_data = []
#     for i, layer in enumerate(layers):
#         sorted_indices = np.argsort(-logit_matrix[i])  # Sort by descending logit value
#         for rank, idx in enumerate(sorted_indices):
#             rank_data.append({
#                 'layer': layer,
#                 'token': tokens[idx],
#                 'rank': rank + 1,
#                 'logit': logit_matrix[i, idx]
#             })
            
#     fig.add_trace(
#         go.Scatter(
#             x=[d['layer'] for d in rank_data],
#             y=[d['rank'] for d in rank_data],
#             mode='markers+text',
#             text=[d['token'] for d in rank_data],
#             textposition="top center",
#             marker=dict(
#                 size=10,
#                 color=[d['logit'] for d in rank_data],
#                 colorscale='Viridis',
#                 showscale=True
#             ),
#             name="Token Rankings"
#         ),
#         row=2, col=1
#     )
    
#     # Update layout
#     fig.update_layout(
#         height=1000,
#         width=1200,
#         title_text=f"Enhanced Logit Lens Analysis" + (f" - Position {position}" if position else ""),
#         showlegend=False,
#     )
    
#     # Update axes
#     fig.update_xaxes(title_text="Tokens", row=1, col=1)
#     fig.update_yaxes(title_text="Layers", row=1, col=1)
#     fig.update_xaxes(title_text="Layers", row=2, col=1)
#     fig.update_yaxes(title_text="Rank", row=2, col=1, autorange="reversed")
    
#     return fig




def plot_enhanced_logit_lens(model_outputs, tokenizer, input_text, position=None):
    """
    Creates a visualization similar to the original logit lens paper style.
    """
    import plotly.graph_objects as go
    import numpy as np
    
    # Extract data
    layers = []
    tokens = []
    logits = []
    
    for layer_idx, layer_data in sorted(model_outputs.items()):
        if not layer_idx.startswith('layer_'):
            continue
            
        layer_num = int(layer_idx.split('_')[1])
        if position is not None:
            token_data = [data for data in layer_data["tokens"].values() 
                         if data["position"] == position][0]
        else:
            token_data = next(iter(layer_data["tokens"].values()))
            
        layer_name = f"h{layer_num}_out" if layer_num > 0 else "h_out"
        layers.append(layer_name)
        
        # Get token predictions and logits
        current_predictions = []
        current_logits = []
        for exp_output in token_data["expert_outputs"]:
            top_tokens = [(tok, prob) for tok, prob in exp_output["top_tokens"]]
            current_predictions.extend([t[0] for t in top_tokens])
            current_logits.extend([t[1] for t in top_tokens])
            
        # Add new unique tokens to the global list
        for tok in current_predictions:
            if tok not in tokens:
                tokens.append(tok)
                
        # Create full logit vector for this layer
        layer_logits = []
        for tok in tokens:
            if tok in current_predictions:
                idx = current_predictions.index(tok)
                layer_logits.append(current_logits[idx])
            else:
                layer_logits.append(0)  # or some small negative number
                
        logits.append(layer_logits)
    
    # Convert to numpy array for easier manipulation
    logit_matrix = np.array(logits)
    
    # Create figure
    fig = go.Figure(data=go.Heatmap(
        z=logit_matrix,
        x=tokens,
        y=layers,
        colorscale=[
            [0, "rgb(0,0,0)"],         # Black for lowest values
            [0.25, "rgb(0,0,128)"],    # Dark blue
            [0.5, "rgb(30,144,255)"],  # Dodger blue
            [0.75, "rgb(211,211,211)"], # Light gray
            [1, "rgb(255,255,0)"]      # Yellow for highest values
        ],
        text=[[f"'{t}'" for t in tokens] for _ in range(len(layers))],
        texttemplate="%{text}",
        textfont={"size": 10, "color": "white"},
        showscale=True,
        colorbar=dict(
            title="Logit Value",
            titleside="right",
            tickfont={"size": 10},
            thickness=15,
            len=0.75
        )
    ))
    
    # Update layout
    fig.update_layout(
        title={
            'text': "Model's Top Token and its Logit",
            'y': 0.95,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': {'size': 14}
        },
        width=1200,
        height=1000,
        xaxis=dict(
            showgrid=False,
            tickangle=45,
            tickfont=dict(size=10),
            tickmode='array',
            ticktext=tokens,
            tickvals=list(range(len(tokens)))
        ),
        yaxis=dict(
            showgrid=False,
            tickfont=dict(size=10),
            autorange='reversed'  # To match the paper's style with h_out at the top
        ),
        plot_bgcolor='white',
        paper_bgcolor='white'
    )
    
    # Add cell borders using shapes
    for i in range(len(layers)+1):
        fig.add_shape(
            type="line",
            x0=-0.5,
            x1=len(tokens)-0.5,
            y0=i-0.5,
            y1=i-0.5,
            line=dict(color="white", width=0.5)
        )
    
    for j in range(len(tokens)+1):
        fig.add_shape(
            type="line",
            x0=j-0.5,
            x1=j-0.5,
            y0=-0.5,
            y1=len(layers)-0.5,
            line=dict(color="white", width=0.5)
        )
    
    return fig

def plot_expert_contributions(model_outputs, position=None):
    """
    Creates visualization of expert contributions at each layer.
    
    Args:
        model_outputs: Dictionary containing layer outputs and expert information
        position: Optional specific position to analyze
    """
    import plotly.graph_objects as go
    import numpy as np
    
    # Extract expert data
    layers = []
    expert_weights = []
    expert_ids = []
    
    for layer_idx, layer_data in sorted(model_outputs.items()):
        if not layer_idx.startswith('layer_'):
            continue
            
        layer_num = int(layer_idx.split('_')[1])
        if position is not None:
            token_data = [data for data in layer_data["tokens"].values() 
                         if data["position"] == position][0]
        else:
            token_data = next(iter(layer_data["tokens"].values()))
            
        layers.append(f"Layer {layer_num}")
        
        # Get expert weights and ids
        layer_weights = []
        layer_ids = []
        for exp_output in token_data["expert_outputs"]:
            layer_weights.append(exp_output["weight"])
            layer_ids.append(exp_output["expert_id"])
            
        expert_weights.append(layer_weights)
        expert_ids.extend([id for id in layer_ids if id not in expert_ids])
        
    # Create matrix of expert weights
    weight_matrix = np.zeros((len(layers), len(expert_ids)))
    for i, weights in enumerate(expert_weights):
        for w, id in zip(weights, expert_ids[:len(weights)]):
            weight_matrix[i, expert_ids.index(id)] = w
            
    # Create figure
    fig = go.Figure(data=go.Heatmap(
        z=weight_matrix,
        x=[f"Expert {id}" for id in expert_ids],
        y=layers,
        colorscale='Viridis',
        text=[[f"{val:.3f}" if val > 0 else "" for val in row] for row in weight_matrix],
        texttemplate="%{text}",
        textfont={"size":10},
        showscale=True,
    ))
    
    # Update layout
    fig.update_layout(
        title=f"Expert Contributions by Layer" + (f" - Position {position}" if position else ""),
        xaxis_title="Experts",
        yaxis_title="Layers",
        height=800,
        width=1200,
    )
    
    return fig

In [8]:
def apply_moe_logit_lens_with_active_expert_outputs(model, inputs, tokenizer, num_active_experts=7):
    model.eval()

    # Initial embedding
    x = model.model.embed_tokens(inputs)

    # Process each layer
    for layer_idx, layer in enumerate(model.model.layers):
        print(f"Layer {layer_idx + 1}")
        x = layer.input_layernorm(x)

        # Self-attention output
        # Remove the position_ids argument here, let the model handle it internally
        attn_output = layer.self_attn(x, x, x)
        x = attn_output + x  # Residual connection
        x = layer.post_attention_layernorm(x)

        # MoE Layer
        moe_output = layer.mlp(x)
        gate_values = moe_output["gate_values"]
        expert_outputs = moe_output["expert_outputs"]

        # Extract active experts
        for batch_idx, gates in enumerate(gate_values):
            active_experts = gates.argsort(descending=True)[:num_active_experts]
            print(f"  Batch {batch_idx + 1}:")
            for expert_idx in active_experts:
                expert_weight = gates[expert_idx].item()
                expert_output = expert_outputs[batch_idx, :, expert_idx]
                top_tokens = expert_output.topk(5, dim=-1)
                top_indices = top_tokens.indices
                top_scores = top_tokens.values
                decoded_tokens = tokenizer.decode(top_indices.tolist())
                print(f"    Expert {expert_idx}: Weight: {expert_weight:.4f}, Tokens: {decoded_tokens}, Scores: {top_scores.tolist()}")

    # Final logits
    logits = model.lm_head(x)
    top_tokens = logits.topk(5, dim=-1)
    decoded_final_tokens = tokenizer.decode(top_tokens.indices.tolist())
    print(f"Final Layer: Tokens: {decoded_final_tokens}, Scores: {top_tokens.values.tolist()}")

In [9]:
def get_device():
    """Get the optimal available device"""
    if torch.cuda.is_available():
        device = torch.device("cuda:0")
        # Enable TF32 for better performance on Ampere GPUs (A100, A6000, etc)
        torch.backends.cuda.matmul.allow_tf32 = True
        torch.backends.cudnn.allow_tf32 = True
        # Set memory allocation settings
        torch.cuda.empty_cache()
        # Enable CUDNN benchmarking for better performance
        torch.backends.cudnn.benchmark = True
    else:
        device = torch.device("cpu")
    return device

In [10]:
def apply_moe_logit_lens_with_active_expert_outputs(model, inputs, tokenizer, num_active_experts=7):
    model.eval()

    # Initial embedding
    x = model.model.embed_tokens(inputs)

    # Process each layer
    for layer_idx, layer in enumerate(model.model.layers):
        print(f"Layer {layer_idx + 1}")
        x = layer.input_layernorm(x)

        # Self-attention output
        # Remove the position_ids argument here, let the model handle it internally
        attn_output = layer.self_attn(x, x, x)
        x = attn_output + x  # Residual connection
        x = layer.post_attention_layernorm(x)

        # MoE Layer
        moe_output = layer.mlp(x)
        gate_values = moe_output["gate_values"]
        expert_outputs = moe_output["expert_outputs"]

        # Extract active experts
        for batch_idx, gates in enumerate(gate_values):
            active_experts = gates.argsort(descending=True)[:num_active_experts]
            print(f"  Batch {batch_idx + 1}:")
            for expert_idx in active_experts:
                expert_weight = gates[expert_idx].item()
                expert_output = expert_outputs[batch_idx, :, expert_idx]
                top_tokens = expert_output.topk(5, dim=-1)
                top_indices = top_tokens.indices
                top_scores = top_tokens.values
                decoded_tokens = tokenizer.decode(top_indices.tolist())
                print(f"    Expert {expert_idx}: Weight: {expert_weight:.4f}, Tokens: {decoded_tokens}, Scores: {top_scores.tolist()}")

    # Final logits
    logits = model.lm_head(x)
    top_tokens = logits.topk(5, dim=-1)
    decoded_final_tokens = tokenizer.decode(top_tokens.indices.tolist())
    print(f"Final Layer: Tokens: {decoded_final_tokens}, Scores: {top_tokens.values.tolist()}")

In [11]:
def logit_lens(model, input_tokens):
    """
    Applies a logit lens to each layer of a mixture of experts (MoE) model for specific input tokens.

    Args:
        model (torch.nn.Module): The MoE model.
        input_tokens (torch.Tensor): Input tensor with token embeddings, shape [batch_size, seq_len].

    Returns:
        dict: A dictionary containing the logit lens outputs for each token at each layer.
    """
    logit_outputs = {}

    # Pass the input tokens through the embedding layer
    embedding_output = model.embed_tokens(input_tokens)  # Shape: [batch_size, seq_len, embedding_dim]

    # Process tokens through each layer
    for layer_idx, layer in enumerate(model.layers):
        # Apply the layer and get its output
        layer_outputs = layer(embedding_output)  # Shape: [batch_size, seq_len, feature_dim]

        # Extract the mixture of experts (MoE) components for this layer
        if hasattr(layer.mlp, "experts"):
            gate_outputs = layer.mlp.gate(embedding_output)  # Shape: [batch_size, seq_len, num_experts]
            expert_outputs = []

            # Process each token separately to extract expert-specific outputs
            for token_idx in range(input_tokens.shape[1]):
                token_expert_outputs = []

                for expert_idx, expert in enumerate(layer.mlp.experts):
                    token_input = embedding_output[:, token_idx, :]  # Shape: [batch_size, embedding_dim]
                    expert_output = expert(token_input)  # Shape: [batch_size, feature_dim]
                    token_expert_outputs.append(expert_output)

                # Stack expert outputs and apply gating
                token_expert_outputs = torch.stack(token_expert_outputs, dim=1)  # Shape: [batch_size, num_experts, feature_dim]
                token_gate_weights = gate_outputs[:, token_idx, :].unsqueeze(-1)  # Shape: [batch_size, num_experts, 1]
                activated_experts_output = torch.sum(token_expert_outputs * token_gate_weights, dim=1)  # Shape: [batch_size, feature_dim]

                # Save activated expert outputs for this token
                logit_outputs[f"layer_{layer_idx}_token_{token_idx}_activated_experts"] = activated_experts_output

        # Update the embedding for the next layer
        embedding_output = layer_outputs

    return logit_outputs

In [12]:
# # Initialize and use as before
# analyzer = MOEExpertLens(model.state_dict(), tokenizer)
# text = "the quick brown fox"
# input_ids = tokenizer(text, return_tensors="pt").input_ids.to(get_device())
# results = analyzer.analyze_text(input_ids)

In [ ]:
# visualize_layer_analysis(tokenizer, results, token_position=3, input_text=text)

In [14]:


text = "the quick brown fox"
inputs = tokenizer(text, return_tensors="pt")
outputs = model.forward(**inputs.to(model.device))
# print(outputs)
x = outputs.logits[0, -1]
x = torch.argmax(x)


output2 = model.generate(**inputs.to(model.device), max_new_tokens=1)

y = output2[0]
# torch.argmax(x)
print(x.shape)
print(y.shape)
result = tokenizer.decode(x)
print(result)

Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48


torch.Size([])
torch.Size([6])
 jumps


In [15]:
outputs = model.forward(**inputs.to(model.device), output_hidden_states=True)


for i in range(29):
    
    x = outputs.hidden_states[i]
    y = x[0, -1]
    z = model.lm_head(y)
    z = torch.argmax(z)
    result = tokenizer.decode(z)
    print(f"Layer {i}: {result}")


Layer 0: oment
Layer 1: ">:
Layer 2: es
Layer 3: es
Layer 4: croft
Layer 5: croft
Layer 6: IEEEeqnarray
Layer 7: IEEEeqnarray
Layer 8: IEEEeqnarray
Layer 9: IEEEeqnarray
Layer 10: IEEEeqnarray
Layer 11: IEEEeqnarray
Layer 12: IEEEeqnarray
Layer 13: IEEEeqnarray
Layer 14: issin
Layer 15: IEEEeqnarray
Layer 16: estrat
Layer 17: IEEEeqnarray
Layer 18: IEEEeqnarray
Layer 19: ="../_
Layer 20:  rejo
Layer 21:  r
Layer 22: Jump
Layer 23: Jump
Layer 24: Jump
Layer 25: Jump
Layer 26: Jump
Layer 27: 

Layer 28:  jumps


In [ ]:
# Create both visualizations
logit_fig = plot_enhanced_logit_lens(results, tokenizer, text, position=0)
expert_fig = plot_expert_contributions(results, position=0)

# Display the figures
logit_fig.show()
expert_fig.show()

In [ ]:
# Create both visualizations
logit_fig = plot_enhanced_logit_lens(results, tokenizer, text, position=1)
expert_fig = plot_expert_contributions(results, position=1)

# Display the figures
logit_fig.show()
expert_fig.show()

In [26]:
# Initialize the lens
lens = MOELens(model, tokenizer)

text_new = "the quick brown fox"
# Analyze text
input_ids = tokenizer(text_new, return_tensors="pt").input_ids.to(model.device)
results = lens.analyze_text(input_ids)

# Clean up hooks when done
lens.remove_hooks()

In [29]:
visualize_layer_analysis(tokenizer, results, token_position=2, input_text=text)
visualize_layer_analysis(tokenizer, results, token_position=4, input_text=text)

Visualizing token:  quick


Visualizing token:  fox
